# Lib

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from imblearn.over_sampling import ADASYN
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from collections import Counter
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import optuna
import shap
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Seed set to {seed}")
set_seed(42)

In [ ]:
train = pd.read_csv('D:/Data Quest Challenge DSI/training_dataset.csv')
test = pd.read_csv('D:/Data Quest Challenge DSI/validation_set.csv')

train.drop(['customer_number'], inplace=True, axis=1)
test.drop(['customer_number'], inplace=True, axis=1)

In [ ]:
print(train.info())
print(test.info())

In [ ]:
# Check for missing values in both train and test datasets
print("Missing values in training dataset:")
print(train.isna().sum())

print("\nMissing values in validation dataset:")
print(test.isna().sum())

# Print total number of rows with missing values in each dataset
print("\nTotal rows with missing values:")
print(f"Training dataset: {train.isna().any(axis=1).sum()} out of {train.shape[0]} rows")
print(f"Validation dataset: {test.isna().any(axis=1).sum()} out of {test.shape[0]} rows")

# EDA

gagal_bayar_sebelumnya cuman ada 2 baris yang classnya yes

In [ ]:
train.head()

there's no class for yes

In [ ]:
test.head()

In [ ]:
cat_col = test.select_dtypes(include=['object']).columns.tolist()
num_col = test.select_dtypes(exclude=['object']).columns.tolist()

print("Categorical columns:", cat_col)
print(f"Jumlah kolom kategorikal: { len(cat_col) }\n")

print("Numerical columns:", num_col)
print("Jumlah kolom numerikal:", len(num_col))

In [ ]:
print("Unique Value pendidikan:", train['pendidikan'].unique())
print("Unique Value pekerjaan:", train['pekerjaan'].unique())
print("Unique Value status_perkawinan:", train['status_perkawinan'].unique())
print("Unique Value bulan_kontak_terakhir:", train['bulan_kontak_terakhir'].unique())
print("Unique Value hari_kontak_terakhir:", train['hari_kontak_terakhir'].unique())
print("Unique Value pulau:", train['pulau'].unique())

# Lookin the distribution of each categorical variable
def plot_categorical_distribution(df, column):
    plt.figure(figsize=(5, 3))
    sns.countplot(data=df, x=column, order=df[column].value_counts().index)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=90)
    plt.show()

plot_categorical_distribution(train, 'pendidikan')
plot_categorical_distribution(train, 'pekerjaan')
plot_categorical_distribution(train, 'status_perkawinan')
plot_categorical_distribution(train, 'bulan_kontak_terakhir')
plot_categorical_distribution(train, 'hari_kontak_terakhir')
plot_categorical_distribution(train, 'pulau')

## Transforming values

Merubah semua kategori unknown menjadi NaN

In [ ]:
# Replace unknown values with NaN
train['pendidikan'] = train['pendidikan'].replace(['unknown'], np.nan)
train['pekerjaan'] = train['pekerjaan'].replace(['unknown'], np.nan)
train['status_perkawinan'] = train['status_perkawinan'].replace(['unknown'], np.nan)
train['gagal_bayar_sebelumnya'] = train['gagal_bayar_sebelumnya'].replace(['unknown'], np.nan)
train['pinjaman_rumah'] = train['pinjaman_rumah'].replace(['unknown'], np.nan)
train['pinjaman_pribadi'] = train['pinjaman_pribadi'].replace(['unknown'], np.nan)
train['pinjaman_pribadi'] = train['pinjaman_pribadi'].replace(['unknown'], np.nan)

test['pendidikan'] = test['pendidikan'].replace(['unknown'], np.nan)
test['pekerjaan'] = test['pekerjaan'].replace(['unknown'], np.nan)
test['status_perkawinan'] = test['status_perkawinan'].replace(['unknown'], np.nan)
test['gagal_bayar_sebelumnya'] = test['gagal_bayar_sebelumnya'].replace(['unknown'], np.nan)
test['pinjaman_rumah'] = test['pinjaman_rumah'].replace(['unknown'], np.nan)
test['pinjaman_pribadi'] = test['pinjaman_pribadi'].replace(['unknown'], np.nan)

# Replace 999 with 0
train['hari_sejak_kontak_sebelumnya'] = train['hari_sejak_kontak_sebelumnya'].replace([999], 0)
test['hari_sejak_kontak_sebelumnya'] = test['hari_sejak_kontak_sebelumnya'].replace([999], 0)

In [ ]:
print("Missing values in training dataset:")
print(train.isna().sum())
print("Duplicate rows in training dataset:")
print(train.duplicated().sum())

In [ ]:
print("\nMissing values in validation dataset:")
print(test.isna().sum())
print("Duplicate rows in validation dataset:")
print(test.duplicated().sum())

# Preprocessing

In [ ]:
train.drop_duplicates(inplace=True)
train.reset_index(drop=True, inplace=True)
print("Duplicate rows in training dataset after removal:")
print(train.duplicated().sum())
print("Duplicate rows in validation dataset after removal:")
print(test.duplicated().sum())

Outliers checking

In [ ]:
# Check for numerical columns 
num_cols = train.select_dtypes(include=['int64', 'float64']).columns

print("Numerical columns in training dataset:", num_cols)

In [ ]:
sns.boxplot(train['jumlah_pekerja'])

In [ ]:
# Threshold atas
def replace_values_above_threshold(data_train, column, threshold):
    sns.boxplot(data_train[column])
    plt.title(f'Original Box Plot of {column}')
    plt.show()

    above_threshold = data_train[column] > threshold
    data_train.loc[above_threshold, column] = threshold

    sns.boxplot(data_train[column])
    plt.title(f'Box Plot with Values Replaced above {threshold}')
    plt.show()

    return train

# train = replace_values_above_threshold(train, 'usia', 68)
# train = replace_values_above_threshold(train, 'jumlah_kontak_kampanye_ini', 6)
# train = replace_values_above_threshold(train, 'indeks_kepercayaan_konsumen', -30)

## Encoding

In [ ]:
le = LabelEncoder()

# One-hot encoding for categorical variables
def one_hot_encode(df, columns):
    return pd.get_dummies(df, columns=columns, drop_first=True)

# Mapping Ordinal Columns
pendidikan_mapping = {
    'TIDAK SEKOLAH': 0,
    'Tidak Tamat SD': 1,
    'SD': 2,
    'SMP': 3,
    'SMA': 4,
    'Diploma': 5,
    'Pendidikan Tinggi': 6,
}

bulan_kontak_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

hasil_mapping = {
    'failure': -1,
    'nonexistent': 0,
    'success': 1,
}

hari_kontak_mapping = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5
}

binary_mapping = {
    'yes': 1,
    'no': 0
}

# train['pendidikan'] = train['pendidikan'].map(pendidikan_mapping)
# test['pendidikan'] = test['pendidikan'].map(pendidikan_mapping)

# train['bulan_kontak_terakhir'] = train['bulan_kontak_terakhir'].map(bulan_kontak_mapping)
# test['bulan_kontak_terakhir'] = test['bulan_kontak_terakhir'].map(bulan_kontak_mapping)

# train['hasil_kampanye_sebelumnya'] = train['hasil_kampanye_sebelumnya'].map(hasil_mapping)
# test['hasil_kampanye_sebelumnya'] = test['hasil_kampanye_sebelumnya'].map(hasil_mapping)

# train['hari_kontak_terakhir'] = train['hari_kontak_terakhir'].map(hari_kontak_mapping)
# test['hari_kontak_terakhir'] = test['hari_kontak_terakhir'].map(hari_kontak_mapping)

# train[['pinjaman_rumah', 'pinjaman_pribadi', 'gagal_bayar_sebelumnya']] = train[['pinjaman_rumah', 'pinjaman_pribadi', 'gagal_bayar_sebelumnya']].replace(binary_mapping)
# test[['pinjaman_rumah', 'pinjaman_pribadi', 'gagal_bayar_sebelumnya']] = test[['pinjaman_rumah', 'pinjaman_pribadi', 'gagal_bayar_sebelumnya']].replace(binary_mapping)

# train = one_hot_encode(train, ['pulau', 'jenis_kontak', 'status_perkawinan'])
# test = one_hot_encode(test, ['pulau', 'jenis_kontak', 'status_perkawinan'])
# train = one_hot_encode(train, ['pekerjaan'])
# test = one_hot_encode(test, ['pekerjaan'])

for col in cat_col:
    if col in train.columns:
        train[col] = le.fit_transform(train[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# train = train.astype('float32')
# test = test.astype('float32')

In [ ]:
train.info()

## Handling missing values

### Dropping

In [ ]:
train_dropped = train.dropna(axis=0, how='any')
print("\nTraining dataset after dropping rows with missing values:")
print(f'Ukuran dataset sebelum drop: {train.shape} dan ukuran dataset setelah drop: {train_dropped.shape}')

test_dropped = test.dropna(axis=0, how='any')
print("\nValidation dataset after dropping rows with missing values:")
print(f'Ukuran dataset sebelum drop: {test.shape} dan ukuran dataset setelah drop: {test_dropped.shape}')

### MICE/KNN

In [ ]:
feature_cols = train_dropped.drop(['berlangganan_deposito'], axis=1).columns.tolist()

In [ ]:
# Initialize imputers
mice = IterativeImputer(random_state=42)
knn = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# Fit MICE imputer on training data and transform both datasets
train_imputed = mice.fit_transform(train[feature_cols])
test_imputed = mice.transform(test)

# Convert back to DataFrame with original column names
train_imputed = pd.DataFrame(train_imputed, columns=train.drop(columns=['berlangganan_deposito']).columns)
train_imputed['berlangganan_deposito'] = train['berlangganan_deposito'].values
test_imputed = pd.DataFrame(test_imputed, columns=test.columns)

# Print shape of imputed datasets
print(f"Training dataset after imputation: {train_imputed.shape}")
print(f"Test dataset after imputation: {test_imputed.shape}")

# Splitting

In [ ]:
X = train.drop(columns=['berlangganan_deposito'])
y = train['berlangganan_deposito']

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
def create_polynomial_features(df, poly):
    poly_features = poly.fit_transform(df.select_dtypes(include=[np.number]))
    poly_feature_names = poly.get_feature_names_out(df.select_dtypes(include=[np.number]).columns)
    poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)
    return poly_df
 
X_poly = poly.fit_transform(X.select_dtypes(include=[np.number]))
X_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.select_dtypes(include=[np.number]).columns))
X_poly = pd.concat([X.drop(columns=X.select_dtypes(include=[np.number]).columns), X_poly], axis=1)

test_poly = poly.transform(test_imputed.select_dtypes(include=[np.number]))
test_poly = pd.DataFrame(test_poly, columns=poly.get_feature_names_out(test_imputed.select_dtypes(include=[np.number]).columns))
test_poly = pd.concat([test_imputed.drop(columns=test_imputed.select_dtypes(include=[np.number]).columns), test_poly], axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_poly, y, test_size=0.25, stratify=y, random_state=42, shuffle=True) 

# Scaling

In [ ]:
minmax = MinMaxScaler(feature_range=(0, 1))
robust = RobustScaler()
X_train_scaled = robust.fit_transform(X_train)
X_val_scaled = robust.transform(X_val)
test_scaled = robust.transform(test_poly)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
test_scaled = pd.DataFrame(test_scaled, columns=test_poly.columns)

X_train_scaled[cat_col] = X_train_scaled[cat_col].astype('int32')
X_val_scaled[cat_col] = X_val_scaled[cat_col].astype('int32')
test_scaled[cat_col] = test_scaled[cat_col].astype('int32')


# Resampling

## Over under

In [ ]:
# # OVER UNDER

# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler

# # Inisialisasi undersampler untuk kelas 0
# undersampler = RandomUnderSampler(sampling_strategy={0: 5000}, random_state=42)
# # Inisialisasi oversampler untuk kelas 1 dan 2
# oversampler = RandomOverSampler(sampling_strategy={1: 2000, 0: 5000}, random_state=42)

# X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# X_train_final, y_train_final = oversampler.fit_resample(X_train_resampled, y_train_resampled)

# from collections import Counter
# print("Distribusi kelas setelah resampling:", Counter(y_train_final['coppaRisk']))

# X_train = X_train_final
# y_train = y_train_final

## SMOTE

In [ ]:
# smote = SMOTE(random_state=42, sampling_strategy='auto')
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
# X_val_resampled, y_val_resampled = smote.fit_resample(X_val_scaled, y_val)
# print("Distribusi kelas setelah SMOTE:")
# print(Counter(y_train_resampled))
# X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
# X_val_resampled = pd.DataFrame(X_val_resampled, columns=X_val.columns)

# Modelling

## Training

In [ ]:
# # Use elasticnet with saga solver and specify l1_ratio
# # For elasticnet, we need to specify both solver='saga' and l1_ratio parameter
# model3 = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)
# model3.fit(X_train_scaled, y_train)

In [ ]:
model2 = CatBoostClassifier(eval_metric='AUC', cat_features=cat_col, random_seed=42, verbose=0)
model2.fit(X_train_scaled, y_train, eval_set=(X_val_scaled, y_val))

In [ ]:
# # Train the model
# model1 = XGBClassifier(objective="binary:logistic", eval_metric="auc", loss='logloss', random_state=42, use_label_encoder=False, verbosity=0)
# model1.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=True)

In [ ]:
# from xgboost import plot_importance
# plt.figure(figsize=(10, 8))
# plot_importance(model1, max_num_features=20, importance_type='weight', title='Feature Importance')
# plt.show()

## Evaluating

In [ ]:
# Make predictions
y_pred = model2.predict(X_val_scaled)
y_pred_proba = model2.predict_proba(X_val_scaled)[:,1]

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)
conf_matrix = confusion_matrix(y_val, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))
print("\nConfusion Matrix:")
print(conf_matrix)

cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['False', 'True'], yticklabels=['False', 'True'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# # Make predictions
# y_pred = model1.predict(X_val_scaled)   
# y_pred_proba = model1.predict_proba(X_val_scaled)[:,1]

# # Evaluate the model
# accuracy = accuracy_score(y_val, y_pred)
# roc_auc = roc_auc_score(y_val, y_pred_proba)
# conf_matrix = confusion_matrix(y_val, y_pred)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"ROC AUC Score: {roc_auc:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_val, y_pred))
# print("\nConfusion Matrix:")
# print(conf_matrix)

# cm = confusion_matrix(y_val, y_pred)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['False', 'True'], yticklabels=['False', 'True'])

In [ ]:
# # Make predictions
# y_pred = model3.predict(X_val_scaled)
# y_pred_proba = model3.predict_proba(X_val_scaled)[:,1]

# # Evaluate the model
# accuracy = accuracy_score(y_val, y_pred)
# roc_auc = roc_auc_score(y_val, y_pred_proba)
# conf_matrix = confusion_matrix(y_val, y_pred)

# print(f"Accuracy: {accuracy:.4f}")
# print(f"ROC AUC Score: {roc_auc:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_val, y_pred))
# print("\nConfusion Matrix:")
# print(conf_matrix)

# cm = confusion_matrix(y_val, y_pred)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['False', 'True'], yticklabels=['False', 'True'])


# Optimizing

In [ ]:
# # Feature importance
# explainer = shap.Explainer(model1)
# shap_values = explainer.shap_values(X_val_scaled)
# shap.summary_plot(shap_values, X_val_scaled, plot_type="bar", max_display=20)

# plot_importance(model1, max_num_features=20, importance_type='weight', title='Feature Importance')


## Optuna

In [ ]:
def objective(trial):
    # Define the hyperparameter search space
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        # 'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS', 'No']),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
        'random_seed': 42,
        'verbose': 0
    }
    
    # First choose boosting type
    param['boosting_type'] = trial.suggest_categorical('boosting_type', ['Ordered', 'Plain'])
    
    # For Ordered boosting, we must use SymmetricTree
    if param['boosting_type'] == 'Ordered':
        param['grow_policy'] = 'SymmetricTree'
    else:
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    # Create model with current hyperparameters
    model = CatBoostClassifier(**param, eval_metric='AUC')
    
    # Train the model
    model.fit(
        X_train_scaled, y_train, 
        eval_set=[(X_val_scaled, y_val)],
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Get validation AUC score
    preds = model.predict_proba(X_val_scaled)[:, 1]
    auc = roc_auc_score(y_val, preds)
    
    return auc

# Create and run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)  # Set timeout to 10 minutes

# Display best parameters and score
print("Best trial:")
print(f"  Value (AUC): {study.best_value:.5f}")
print("  Params:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

# Train final model with best parameters
best_params = study.best_params
best_params['random_seed'] = 42
best_params['verbose'] = 0
best_params['eval_metric'] = 'AUC'

best_model = CatBoostClassifier(**best_params)
best_model.fit(
    X_train_scaled, y_train, 
    eval_set=[(X_val_scaled, y_val)],
    early_stopping_rounds=50,
    verbose=False
)

# Evaluate the optimized model
y_pred = best_model.predict(X_val_scaled)
y_pred_proba = best_model.predict_proba(X_val_scaled)[:, 1]
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)

print(f"\nOptimized Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# Plot confusion matrix
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
            xticklabels=['False', 'True'], yticklabels=['False', 'True'])
plt.title('Confusion Matrix - Optimized CatBoost Model')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Feature importance
feature_importance = best_model.get_feature_importance()
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
plt.title('Top 20 Feature Importance - Optimized CatBoost Model')
plt.tight_layout()
plt.show()

# Save the best model for later prediction
model2_optimized = best_model

## RFE

In [ ]:
# rfe = RFE(estimator=model2_optimized, step=1)
# rfe.fit(X_train_scaled, y_train)
# rfe_support = rfe.support_
# rfe_ranking = rfe.ranking_
# rfe_features = X_train_scaled.columns[rfe_support]
# print("RFE Selected Features:")
# print(rfe_features.tolist())
# # Create a new DataFrame with RFE selected features
# X_train_rfe = X_train_scaled[rfe_features]
# X_val_rfe = X_val_scaled[rfe_features]
# test_rfe = test_scaled[rfe_features]

# X_train_rfe

In [ ]:
# model_rfe = model2_optimized
# model_rfe.fit(X_train_rfe, y_train, eval_set=(X_val_rfe, y_val), early_stopping_rounds=50, verbose=False)
# # Evaluate the RFE model
# y_pred_rfe = model_rfe.predict(X_val_rfe)
# y_pred_proba_rfe = model_rfe.predict_proba(X_val_rfe)[:, 1]
# accuracy_rfe = accuracy_score(y_val, y_pred_rfe)
# roc_auc_rfe = roc_auc_score(y_val, y_pred_proba_rfe)

# print(f"RFE Model Performance:")
# print(f"Accuracy: {accuracy_rfe:.4f}")
# print(f"ROC AUC Score: {roc_auc_rfe:.4f}")

# # Plot confusion matrix for RFE model
# cm_rfe = confusion_matrix(y_val, y_pred_rfe)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_rfe, annot=True, fmt='d', cmap='Blues', cbar=False, 
#             xticklabels=['False', 'True'], yticklabels=['False', 'True'])
# plt.title('Confusion Matrix - RFE CatBoost Model')
# plt.xlabel('Predicted')
# plt.ylabel('Actual')

# Predict

In [ ]:
# save the model
model2_optimized.save_model('catboost_model_rfe3.cbm')

# Save the parameters
with open('catboost_params_rfe3.txt', 'w') as f:
    for key, value in best_params.items():
        f.write(f"{key}: {value}\n")


In [ ]:
# Id Column
test_id = pd.read_csv('D:/Data Quest Challenge DSI/validation_set.csv')['customer_number']

# Make predictions on the test set
test_predictions = model2_optimized.predict_proba(test_scaled)[:, 1]

# Create submission DataFrame
submission = pd.DataFrame({
    'customer_number': test_id,
    'berlangganan_deposito': test_predictions
})

# Save submission to CSV
submission.to_csv('submission4.csv', index=False)
print("Submission file created successfully!")